In [9]:
import sys
sys.path.append('..')
from gamadhani.utils.utils import download_models, download_data
from gamadhani.src.dataset import SequenceDataset
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd

from gamadhani.utils.generate_utils import load_audio_fns
from gamadhani.utils.utils import get_device
import gin
from functools import partial
import torch
import gamadhani.utils.pitch_to_audio_utils as p2a

from torch import nn

In [4]:
device = get_device()
_, _, audio_model_path, audio_qt_path = download_models('kmaneeshad/GaMaDHaNi', pitch_model_type="diffusion")
audio_model, audio_qt, audio_seq_len, invert_audio_fn = load_audio_fns(audio_path=audio_model_path,
    qt_path=audio_qt_path,
    config_path='/home/mila/n/nithya.shikarpur/GaMaDHaNi-dev/configs/pitch_to_audio_config.gin',
device=device)

Script is running on: GPU


/home/mila/n/nithya.shikarpur/GaMaDHaNi-dev/quantization-studies/../gamadhani/utils/generate_utils.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckp

In [12]:
gin.parse_config_file('/home/mila/n/nithya.shikarpur/GaMaDHaNi-dev/configs/pitch_to_audio_config_updated.gin')
db = '/home/mila/n/nithya.shikarpur/scratch/final-data-ismir/data/merged_data-finalest/cached-audio-pitch-16k/train'
db = SequenceDataset(db)

In [13]:
db[0]

TypeError: load_cached_dataset() got an unexpected keyword argument 'kwargs'
  In call to configurable 'load_cached_dataset' (<function load_cached_dataset at 0x7f90e9fd20e0>)